# Predicción de demanda con Random Forest

In [63]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 


from sklearn.preprocessing import scale
import timeit

# Lectura de datos

- **ant**: antelación hasta el vuelo.

- **ruta**: trayecto de ida y vuelta ordenado alfabéticamente.
        
- **aeropuerto_origen**: aeropuerto donde despega el avión.

- **aeropuerto_destino**: aeropuerto donde aterriza el avión.
    
- **fecha_salida**: fecha de despegue del avión.
    
- **num_vuelo_operador**: identificador de número de vuelo.

- **month**: mes de vuelo.

- **weekday**: día de la semana de vuelo.
    
- **timezone**: franja horaria de vuelo.
    
- **year**: año de vuelo.
    
- **capacidad**: número máximo de pax en cada vuelo, puede variar en cada vuelo.
    
- **demand**: billetes vendidos en cada vuelo.
    
- **first**: código del principal competidor de la ruta del vuelo.
    
- **second**: código del segundo competidor de la ruta del vuelo.
    
- **first_ratio**: 
    
- **second_ratio**:

- **hit**:

- **first_weight**: peso de importancia del primer competidor en la ruta del vuelo respecto a I2.

- **second_weight**: peso de importancia del segundo competidor en la ruta del vuelo respecto a I2.

- **global_first_weight**: 

- **global_second_weight**: 

- **first_p**:

- **second_p**:

In [64]:
# Load the dataset 
datos = pd.read_csv("../datasets/datos_pred_demanda.csv", sep=';', decimal=',')
datos.head()

,ant,ruta,aeropuerto_origen,aeropuerto_destino,fecha_salida,num_vuelo_operador,month,weekday,timezone,year,...,second,first_ratio,second_ratio,hit,first_weight,second_weight,global_first_ratio,global_second_ratio,first_p,second_p
0,11,ACEMAD,ACE,MAD,2017-02-02,3857,2,Thursday,Mediodia,2017,...,UX,3.140340,0.000000,2,0.0,0.0,2.520623,0.000000,0.623683,0.000000
1,11,ACEMAD,ACE,MAD,2017-02-03,3857,2,Friday,Mediodia,2017,...,UX,3.610000,0.000000,2,0.0,0.0,2.210075,0.000000,0.846071,0.000000
2,11,ACEMAD,ACE,MAD,2017-02-04,3857,2,Saturday,Mediodia,2017,...,UX,0.000000,3.252252,1,0.0,1.0,0.000000,1.595346,0.000000,0.981601
3,11,ACEMAD,ACE,MAD,2017-02-05,3857,2,Sunday,Mediodia,2017,...,UX,3.094286,3.252252,2,0.0,1.0,0.000000,1.485467,0.658558,0.955942
4,11,ACEMAD,ACE,MAD,2017-02-06,3857,2,Monday,Mediodia,2017,...,UX,2.022857,0.000000,2,0.0,0.0,2.648854,0.000000,0.387667,0.000000


In [65]:
datos.shape

(625798, 24)

In [66]:
datos['ruta'].unique()

array(['ACEMAD', 'LGWMAD', 'MADPMI', 'MADSCQ', 'MADTXL'], dtype=object)

Tenemos casi un millón de patrones correspondientes a datos de 5 rutas diferentes y 24 características o variables.

Seleccionamos datos de antelacion 11 para tener un conjunto de datos asequible para esta clase.

In [67]:
# seleccionamos datos de antelacion 11 para tener un conjunto de datos asequible para esta clase
datos = datos[datos.ant==11]

In [68]:
datos.shape

(34242, 24)

# Tratamiento de datos

Vamos a ordenar los datos por fecha salida para cuando escojamos el conjunto de train, validation y test se respete el eje temporal

In [69]:
datos = datos.sort_values(by=['fecha_salida'], ascending=True)

Borramos las variables que no aportan información al modelo

In [70]:
del datos["ant"]
del datos["num_vuelo_operador"]

Transformar variables categoricas en numericas.

1) Primero definimos las variables numericas y categoricas.

2) Aplicamos one hot enconding

In [71]:
categorical_vars = ['ruta', 'aeropuerto_origen', 'aeropuerto_destino', 'month', 'weekday', 'year', 
                    'nombre_blackout', 'first', 'second', 'timezone', 'hit']

categorical_vars

['ruta',
 'aeropuerto_origen',
 'aeropuerto_destino',
 'month',
 'weekday',
 'year',
 'nombre_blackout',
 'first',
 'second',
 'timezone',
 'hit']

In [72]:
numerical_vars = list(set(datos.columns) - set(categorical_vars))
numerical_vars

['global_first_ratio',
 'second_weight',
 'fecha_salida',
 'demand',
 'first_p',
 'first_weight',
 'first_ratio',
 'global_second_ratio',
 'second_p',
 'second_ratio',
 'capacidad']

## One hot encoding

Importamos la libreria que hace one hot encoding

In [73]:
from sklearn.preprocessing import OneHotEncoder

Realizamos one hot encoding de las variables categoricas

In [74]:
ohe = OneHotEncoder(sparse = False)

ohe_fit = ohe.fit(datos[categorical_vars])
X_ohe = pd.DataFrame(ohe.fit_transform(datos[categorical_vars]))
X_ohe.columns = pd.DataFrame(ohe_fit.get_feature_names())


Visualizamos los datos iniciales y los que están con one hot encoding para ver la diferencia

In [75]:
datos.head()

,ruta,aeropuerto_origen,aeropuerto_destino,fecha_salida,month,weekday,timezone,year,capacidad,demand,...,second,first_ratio,second_ratio,hit,first_weight,second_weight,global_first_ratio,global_second_ratio,first_p,second_p
0,ACEMAD,ACE,MAD,2017-02-02,2,Thursday,Mediodia,2017,213,167,...,UX,3.140340,0.000000,2,0.0,0.0,2.520623,0.000000,0.623683,0.000000
29080,MADTXL,MAD,TXL,2017-02-02,2,Thursday,Tarde,2017,176,116,...,UNKNOWN,2.557941,0.000000,1,1.0,0.0,2.415747,0.000000,0.640143,0.000000
6007,LGWMAD,MAD,LGW,2017-02-02,2,Thursday,Matutina,2017,172,106,...,D8,1.000000,0.000000,1,1.0,0.0,1.339117,0.000000,0.405146,0.000000
6008,LGWMAD,MAD,LGW,2017-02-02,2,Thursday,Tarde_Noche,2017,176,71,...,D8,0.000000,0.845015,1,0.0,1.0,0.000000,1.013701,0.000000,0.275469
8192,MADPMI,MAD,PMI,2017-02-02,2,Thursday,Tarde,2017,174,112,...,UX,0.000000,1.813973,1,0.0,1.0,0.000000,1.149599,0.000000,0.991889


In [76]:
X_ohe.head()

,"(x0_ACEMAD,)","(x0_LGWMAD,)","(x0_MADPMI,)","(x0_MADSCQ,)","(x0_MADTXL,)","(x1_ACE,)","(x1_LGW,)","(x1_MAD,)","(x1_PMI,)","(x1_SCQ,)",...,"(x9_Matutina,)","(x9_Mediodia,)","(x9_Noche,)","(x9_Nocturna,)","(x9_Tarde,)","(x9_Tarde_Noche,)","(x10_1,)","(x10_2,)","(x10_3,)","(x10_5,)"
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


Concatenamos los datos categoricos con one hot enconding y los numericos para tener el dataset completo

In [77]:
datos = pd.concat((X_ohe, datos[numerical_vars].reset_index()), axis=1)
del datos['index']

In [78]:
datos.columns[datos.dtypes == object]

Index(['fecha_salida'], dtype='object')

Comprobamos que ahora todos los datos son numericos menos fecha salida, que no entrará al modelo.

# Escalado

In [79]:
fecha_salida_values = datos['fecha_salida']
y = datos['demand']
del datos['fecha_salida']
del datos['demand']

Escalamos los datos mediante la media y la desviación típica.

Este paso es muy importante en un modelo machine learning ya que si las variables tienen escalas muy diferentes el modelo dará mucho más peso a las variables con valores mas altos.

In [80]:
datos_scale = pd.DataFrame(scale(datos))
datos_scale.columns = datos.columns

Visualizamos los datos iniciales y los escalados para ver la diferencia.

In [81]:
print(datos.head())
print(datos_scale.head())

   (x0_ACEMAD,)  (x0_LGWMAD,)  (x0_MADPMI,)  (x0_MADSCQ,)  (x0_MADTXL,)  \
0           1.0           0.0           0.0           0.0           0.0   
1           0.0           0.0           0.0           0.0           1.0   
2           0.0           1.0           0.0           0.0           0.0   
3           0.0           1.0           0.0           0.0           0.0   
4           0.0           0.0           1.0           0.0           0.0   

   (x1_ACE,)  (x1_LGW,)  (x1_MAD,)  (x1_PMI,)  (x1_SCQ,)  ...  (x10_5,)  \
0        1.0        0.0        0.0        0.0        0.0  ...       0.0   
1        0.0        0.0        1.0        0.0        0.0  ...       0.0   
2        0.0        0.0        1.0        0.0        0.0  ...       0.0   
3        0.0        0.0        1.0        0.0        0.0  ...       0.0   
4        0.0        0.0        1.0        0.0        0.0  ...       0.0   

   global_first_ratio  second_weight   first_p  first_weight  first_ratio  \
0            2.520623

In [82]:
datos = datos_scale
datos['fecha_salida'] = fecha_salida_values
datos['demand'] = y

# Elección de los conjuntos de entrenamiento

1.1 Conjunto de Validación Fijo

Utilizaremos a modo de ejemplo los ratios habitualmente recomendados:

• Train: 70%.

• Validación: 15%.

• Test: 15%.


In [83]:
perc_values = [0.7, 0.15, 0.15];

In [84]:
# Selección del patrón de datos X y del target y
y = datos['demand']
X = datos.loc[:, datos.columns != 'demand']

Creamos los conjuntos de train, validacion y test con el tamaño seleccionado pero respetando el eje temporal

In [85]:
# dimensiones de los conjuntos de train y test
n_train = int(X.shape[0] * perc_values[0])
n_val = int(X.shape[0] * perc_values[1])
n_test = int(X.shape[0] * perc_values[2])

# selección del conjunto de train
X_train = X.iloc[:n_train]
y_train = y.iloc[:n_train]

# selección del conjunto de validación
X_val = X.iloc[(n_train):(n_train+n_val)]
y_val = y.iloc[(n_train):(n_train+n_val)]

# selección del conjunto de test
X_test = X.iloc[(n_train+n_val):]
y_test = y.iloc[(n_train+n_val):]

Visualizamos el tamaño de los conjuntos para el set con el eje temporal

In [86]:
print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))
print('Validation data size = ' + str(X_val.shape))
print('Validation target size = ' + str(y_val.shape))
print('Test data size = ' + str(X_test.shape))
print('Test target size = ' + str(y_test.shape))

Train data size = (23969, 73)
Train target size = (23969,)
Validation data size = (5136, 73)
Validation target size = (5136,)
Test data size = (5137, 73)
Test target size = (5137,)


Visualizamos las fecha de salida de los conjuntos para el set con el eje temporal

In [87]:
# eje temporal de cada conjunto
print('Train min date data = ' + np.amin(X_train["fecha_salida"]))
print('Train max date data = ' + np.amax(X_train["fecha_salida"]))

print('Val min date data = ' + np.amin(X_val["fecha_salida"]))
print('Val max date data = ' + np.amax(X_val["fecha_salida"]))

print('Test min date data = ' + np.amin(X_test["fecha_salida"]))
print('Test max date data = ' + np.amax(X_test["fecha_salida"]))

Train min date data = 2017-02-02
Train max date data = 2019-05-02
Val min date data = 2019-05-03
Val max date data = 2019-09-19
Test min date data = 2019-09-19
Test max date data = 2020-02-18


Una vez hechas las comprobaciones borramos la variable fecha_salida ya que no es una variable del modelo.

In [88]:
del X_train['fecha_salida']
del X_val['fecha_salida']
del X_test['fecha_salida']


# Modelos

Scikit-Learn dispone de una extensa batería de modelos de Machine Learning disponible, siendo esta una de las grandes ventajas que tiene el uso de esta librería.

Otra de las razones de su popularidad es que proporciona un framework de uso que es general para todos los tipos de modelos. Básicamente consiste de 6 pasos:

1.Importar modelo que se quiere emplear. Ver lista de modelos supervisados en: https://scikit-learn.org/stable/supervised_learning.html


2.Importa métrica a emplear. Ver lista de métricas disponibles en https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.


3.Definir modelo.


4.Llamar al método fit para entrenar el modelo.


5.Llamar al método predict para generar las predicciones.


6.Calcular métrica usando las predicciones obtenidas en el paso anterior.


Veamos todos estos pasos con un ejemplo sencillo



1) Importar modelo, en este caso Árboles de decisión para regresión


In [89]:
from sklearn.tree import DecisionTreeRegressor  

2) Importar métrica, en este caso MAE y MSE

In [90]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

3) Definir el método con los parámetros por defecto

In [91]:
 model = DecisionTreeRegressor(random_state = 0)

4) Llamar al método fit para entrenar el modelo

Añadimos la función timeit para medir el tiempo que tarda el modelo.

De esta manera podremos hacer una estimación de cuanto tardará nuestro grid. Aunque esto dependerá de muchas otras cosas, por ejemplo, como la profundidad del árbol.

In [92]:
## Ponemos un contador para ver cuanto tarda cada modelo
start = timeit.default_timer()

model.fit(X_train, np.array(y_train))

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  0.25861699999995835


5) Llamar al método predict para generar las predicciones.

In [93]:
start = timeit.default_timer()

pred_train = model.predict(X_train)
pred_val = model.predict(X_val)
pred_test = model.predict(X_test)

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  0.01830619999998362


In [94]:
# Comprobamos las primeras predicciones
print(pred_train[0:5])
print(y_train[0:5])

[167. 116. 106.  71. 112.]
0    167
1    116
2    106
3     71
4    112
Name: demand, dtype: int64


6) Calcular métrica usando las predicciones obtenidas en el paso anterior.

In [95]:
# Calcular métricas de evaluación
mae_train = mae(y_train, pred_train);
mae_val = mae(y_val, pred_val);
mae_test = mae(y_test, pred_test);

mse_train = mse(y_train, pred_train);
mse_val = mse(y_val, pred_val);
mse_test = mse(y_test, pred_test);

In [96]:
print('Train: MAE = ' + str(mae_train) + ' - MSE = '  + str(mse_train))
print('Validation: MAE = ' + str(mae_val) + ' - MSE = '  + str(mse_val))
print('Test: MAE = ' + str(mae_test) + ' - MSE = '  + str(mse_test))

Train: MAE = 0.1705814454781871 - MSE = 2.8999506306757334
Validation: MAE = 22.441783489096572 - MSE = 867.1947040498443
Test: MAE = 24.9263188631497 - MSE = 1060.9363928362857


Calculamos una nueva métrica el MAPE, que tienen en cuenta el rango del target.

In [97]:
def mape(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred)/y_pred) * 100

In [98]:
# Calcular métricas de evaluación
mape_train = mape(y_train, pred_train);
mape_val = mape(y_val, pred_val);
mape_test = mape(y_test, pred_test);

In [99]:
print('Train: MAPE = ' + str(mape_train))
print('Validation: MAPE = ' + str(mape_val))
print('Test: MAPE = ' + str(mape_test))

Train: MAPE = 0.15192107389593038
Validation: MAPE = 19.786244900668425
Test: MAPE = 22.41531720964626


## Variable Importance

Vamos a calcular las variables más importantes para nuestro modelo anterior.

In [107]:
model?

In [105]:
varimp = pd.DataFrame({'variable' : X_train.columns, 'imp' : model.feature_importances_})
varimp = varimp.sort_values(by = 'imp', ascending=False)
varimp

,variable,imp
71,capacidad,0.259686
70,second_ratio,0.073062
63,global_first_ratio,0.062748
67,first_ratio,0.059548
65,first_p,0.049015
...,...,...
46,"(x7_D8,)",0.000436
50,"(x8_UNKNOWN,)",0.000391
43,"(x6_puente de verano,)",0.000283
56,"(x9_Nocturna,)",0.000132


Probemos a seleccionar de nuestro dataset únicamente el **TOP 20** de variables más importantes y entrenar de nuevo el modelo.

In [148]:
selected_variables = varimp.variable.values[0:20]
X_train_new = X_train[selected_variables]
X_val_new = X_val[selected_variables]
X_test_new = X_test[selected_variables]
X_train_new

,capacidad,second_ratio,global_first_ratio,first_ratio,first_p,second_p,global_second_ratio,"(x0_MADSCQ,)","(x9_Matutina,)","(x3_1,)","(x9_Noche,)","(x4_Sunday,)",first_weight,"(x5_2017,)","(x3_12,)","(x6_NO,)",second_weight,"(x6_verano,)","(x3_3,)","(x3_2,)"
0,2.181833,-0.748274,1.116446,1.229701,0.920076,-0.795307,-0.784568,-0.596127,-0.550651,-0.303897,-0.354511,-0.409576,-0.836621,1.645799,-0.304642,0.708641,-0.828474,-0.450407,-0.285636,3.418614
1,-0.400404,-0.748274,1.029807,0.845664,0.970004,-0.795307,-0.784568,-0.596127,-0.550651,-0.303897,-0.354511,-0.409576,1.566123,1.645799,-0.304642,0.708641,-0.828474,-0.450407,-0.285636,3.418614
2,-0.679565,-0.748274,0.140395,-0.181651,0.257213,-0.795307,-0.784568,-0.596127,1.816034,-0.303897,-0.354511,-0.409576,1.566123,1.645799,-0.304642,0.708641,-0.828474,-0.450407,-0.285636,3.418614
3,-0.400404,0.311369,-0.965861,-0.841056,-0.971669,0.035931,0.781760,-0.596127,-0.550651,-0.303897,-0.354511,-0.409576,-0.836621,1.645799,-0.304642,0.708641,1.584820,-0.450407,-0.285636,3.418614
4,-0.539984,1.526437,-0.965861,-0.841056,-0.971669,2.197753,0.991742,-0.596127,-0.550651,-0.303897,-0.354511,-0.409576,-0.836621,1.645799,-0.304642,0.708641,1.584820,-0.450407,-0.285636,3.418614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23964,2.042253,0.230366,-0.965861,-0.841056,-0.971669,0.689179,1.130443,-0.596127,-0.550651,-0.303897,-0.354511,-0.409576,-0.836621,-0.607608,-0.304642,-1.411153,1.584820,-0.450407,-0.285636,-0.292516
23965,-0.400404,-0.748274,0.020109,0.076169,0.153562,-0.795307,-0.784568,1.677495,1.816034,-0.303897,-0.354511,-0.409576,1.566123,-0.607608,-0.304642,-1.411153,-0.828474,-0.450407,-0.285636,-0.292516
23966,-0.400404,-0.748274,0.919075,1.390297,0.656810,-0.795307,-0.784568,-0.596127,1.816034,-0.303897,-0.354511,-0.409576,1.566123,-0.607608,-0.304642,-1.411153,-0.828474,-0.450407,-0.285636,-0.292516
23967,-0.400404,-0.748274,1.321774,-0.135126,-0.619804,-0.795307,-0.784568,-0.596127,1.816034,-0.303897,-0.354511,-0.409576,1.566123,-0.607608,-0.304642,-1.411153,-0.828474,-0.450407,-0.285636,-0.292516


In [149]:
model.fit(X_train_new, np.array(y_train))

DecisionTreeRegressor(random_state=0)

In [150]:
# Calcular predicciones
pred_train = model.predict(X_train_new)
pred_val = model.predict(X_val_new)

In [151]:
# Calcular métricas de evaluación
mape_train = mape(y_train, pred_train);
mape_val = mape(y_val, pred_val);

In [152]:
print('Validation: MAPE = ' + str(mape_val))

Validation: MAPE = 18.60683081299571


Hemos mejorar nuestro error de validación.

## Grid Search

Vamos a proceder a calcular los parámetros óptimos para nuestro modelo

Definimos los paramétros que queremos optimizar en este caso:

- **criterion (por defecto giny)**: criterio utilizado como regla de división. Podría ser entropy.

<img src="../figures/formulas_gini_entropia.PNG" width="50%">

- **min_samples_split (por defecto 2)**: se refiere a la cantidad mínima de muestras que debe tener un nodo para poder subdividir.

- **min_samples_leaf (por defecto 1)**:  cantidad mínima que puede tener una hoja final. Si tuviera menos, no se formaría esa hoja y «subiría» un nivel, su antecesor.

- **max_features (por defecto None)**: representan el número de variables para elegir el mejor split. Si se pone a None max_features=n_features. 

        - If int, then consider max_features features at each split.
        - If float, then max_features is a fraction and int(max_features * n_features) features are considered at each split.
        - If “auto”, then max_features=sqrt(n_features).
        - If “sqrt”, then max_features=sqrt(n_features).
        - If “log2”, then max_features=log2(n_features).
        - If None, then max_features=n_features.

- **max_depth (por defecto None)** Es la profundidad máxima del árbol, esto nos permite controlar el crecimiento y que el árbol no se haga muy complejo. Si el parámetro es None el árbol se expandirá hasta que todas las hojas son puras o se cumple el min_samples_split, es decir, tiene menos patrones que los que hemos definido.


Definimos nuestra rejilla con los parámetros anteriores

# Exhaustive Grid Search

In [38]:

min_samples_splits = np.linspace(0.1, 1.0, 5, endpoint=True)
min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
max_depths = np.linspace(1, 32, 32, endpoint=True)

param_grid = [
  {'criterion': ["mse", "mae"], 'min_samples_split': min_samples_splits,'min_samples_leaf': min_samples_leafs,  
   'max_features': ["sqrt","log2"], 'max_depth': max_depths}
 ]

param_grid

[{'criterion': ['mse', 'mae'],
  'min_samples_split': array([0.1  , 0.325, 0.55 , 0.775, 1.   ]),
  'min_samples_leaf': array([0.1, 0.2, 0.3, 0.4, 0.5]),
  'max_features': ['sqrt', 'log2'],
  'max_depth': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
         27., 28., 29., 30., 31., 32.])}]

Escogemos la lista de parametros de param_grid

In [39]:
params_values = param_grid[0]
params_values

{'criterion': ['mse', 'mae'],
 'min_samples_split': array([0.1  , 0.325, 0.55 , 0.775, 1.   ]),
 'min_samples_leaf': array([0.1, 0.2, 0.3, 0.4, 0.5]),
 'max_features': ['sqrt', 'log2'],
 'max_depth': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32.])}

Calculamos el numero de iteraciones totales como la multiplicación de la longitud de los posibles valores de todos los hiperaparámetros.

In [40]:
num_iteraciones = len(params_values['criterion'])*len(params_values['min_samples_split'])*len(params_values['min_samples_leaf'])*len(params_values['max_features'])*len(params_values['max_depth'])

print('El numero de iteraciones es', num_iteraciones)

El numero de iteraciones es 3200


In [41]:
metric = mae
error_val = []
error_val= pd.DataFrame(columns=('val_error', 'criterion', 'min_samples_split', 'min_samples_leaf', 'max_features', 'max_depth'))
num_iter = 0

In [42]:
astart = timeit.default_timer()

for a in range(0,len(params_values['criterion'])):
    for b in range(0,len(params_values['min_samples_split'])):
        for c in range(0,len(params_values['min_samples_leaf'])):
            for d in range(0,len(params_values['max_features'])):
                for e in range(0,len(params_values['max_depth'])):
                    # print control iteracion modelo
                    print('Numero de iteración = ' + str(num_iter) +
                          ', parametro criterio = ' + str(params_values['criterion'][a]) + 
                          ', parametro min_samples_split = '  + str(params_values['min_samples_split'][b]) +
                          ', parametro min_samples_leaf = '  + str(params_values['min_samples_leaf'][c]) +
                          ', parametro max_features = '  + str(params_values['max_features'][d]) +
                          ', parametro profundidad maxima = '  + str(params_values['max_depth'][e]))

                    # definicion del modelo con sus parametros
                    model = DecisionTreeRegressor(criterion = params_values['criterion'][a],
                                                  min_samples_split = params_values['min_samples_split'][b],
                                                  min_samples_leaf = params_values['min_samples_leaf'][c],
                                                  max_features = params_values['max_features'][d],
                                                  max_depth = params_values['max_depth'][e],
                                                 random_state = 1)

                    # entrenamiento del modelo
                    model.fit(X_train, np.array(y_train))

                    # prediccion del conjunto de validacion
                    pred_val = model.predict(X_val)

                    # Calculo de la metrica de error
                    error_val_iter = metric(y_val, pred_val)

                    # print error
                    print('Error validacion = ' + str(error_val_iter))

                    # guarda el error
                    error_val.loc[num_iter]=[error_val_iter,
                                            params_values['criterion'][a], 
                                            params_values['min_samples_split'][b],
                                            params_values['min_samples_leaf'][c],
                                            params_values['max_features'][d],
                                            params_values['max_depth'][e]] 
                    num_iter += 1

stop = timeit.default_timer()
print('Time: ', stop - start) 

Numero de iteración = 0, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 1.0
Error validacion = 20.76490024632162
Numero de iteración = 1, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 2.0
Error validacion = 20.65605762256878
Numero de iteración = 2, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 3.0
Error validacion = 20.54035066426064
Numero de iteración = 3, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 4.0
Error validacion = 20.638956198207595
Numero de iteración = 4, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples

Error validacion = 24.240495024378003
Numero de iteración = 40, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = log2, parametro profundidad maxima = 9.0
Error validacion = 24.240495024378003
Numero de iteración = 41, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = log2, parametro profundidad maxima = 10.0
Error validacion = 24.240495024378003
Numero de iteración = 42, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = log2, parametro profundidad maxima = 11.0
Error validacion = 24.240495024378003
Numero de iteración = 43, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = log2, parametro profundidad maxima = 12.0
Error validacion = 24.240495024378003
Numero de iteración = 44, parametro criterio = mse, paramet

Error validacion = 20.855551796761514
Numero de iteración = 79, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = sqrt, parametro profundidad maxima = 16.0
Error validacion = 20.855551796761514
Numero de iteración = 80, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = sqrt, parametro profundidad maxima = 17.0
Error validacion = 20.855551796761514
Numero de iteración = 81, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = sqrt, parametro profundidad maxima = 18.0
Error validacion = 20.855551796761514
Numero de iteración = 82, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = sqrt, parametro profundidad maxima = 19.0
Error validacion = 20.855551796761514
Numero de iteración = 83, parametro criterio = mse, parame

Numero de iteración = 119, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = log2, parametro profundidad maxima = 24.0
Error validacion = 24.338297313153518
Numero de iteración = 120, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = log2, parametro profundidad maxima = 25.0
Error validacion = 24.338297313153518
Numero de iteración = 121, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = log2, parametro profundidad maxima = 26.0
Error validacion = 24.338297313153518
Numero de iteración = 122, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.2, parametro max_features = log2, parametro profundidad maxima = 27.0
Error validacion = 24.338297313153518
Numero de iteración = 123, parametro criterio = mse, parametro min_samples_split = 0.1, para

Error validacion = 20.76490024632162
Numero de iteración = 156, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.30000000000000004, parametro max_features = sqrt, parametro profundidad maxima = 29.0
Error validacion = 20.76490024632162
Numero de iteración = 157, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.30000000000000004, parametro max_features = sqrt, parametro profundidad maxima = 30.0
Error validacion = 20.76490024632162
Numero de iteración = 158, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.30000000000000004, parametro max_features = sqrt, parametro profundidad maxima = 31.0
Error validacion = 20.76490024632162
Numero de iteración = 159, parametro criterio = mse, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.30000000000000004, parametro max_features = sqrt, parametro profundidad maxima = 32.0
Error validacion = 20.7649002463

...

Numero de iteración = 1502, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.4, parametro max_features = sqrt, parametro profundidad maxima = 31.0
Error validacion = 24.399172925404407
Numero de iteración = 1503, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.4, parametro max_features = sqrt, parametro profundidad maxima = 32.0
Error validacion = 24.399172925404407
Numero de iteración = 1504, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.4, parametro max_features = log2, parametro profundidad maxima = 1.0
Error validacion = 24.399172925404407
Numero de iteración = 1505, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.4, parametro max_features = log2, parametro profundidad maxima = 2.0
Error validacion = 24.399172925404407
Numero de iteración = 1506, parametro criterio = mse, parametro min_samples_split = 1.0, p

Numero de iteración = 1552, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = sqrt, parametro profundidad maxima = 17.0
Error validacion = 24.505153625873717
Numero de iteración = 1553, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = sqrt, parametro profundidad maxima = 18.0
Error validacion = 24.505153625873717
Numero de iteración = 1554, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = sqrt, parametro profundidad maxima = 19.0
Error validacion = 24.505153625873717
Numero de iteración = 1555, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = sqrt, parametro profundidad maxima = 20.0
Error validacion = 24.505153625873717
Numero de iteración = 1556, parametro criterio = mse, parametro min_samples_split = 1.0,

Numero de iteración = 1592, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = log2, parametro profundidad maxima = 25.0
Error validacion = 24.505153625873717
Numero de iteración = 1593, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = log2, parametro profundidad maxima = 26.0
Error validacion = 24.505153625873717
Numero de iteración = 1594, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = log2, parametro profundidad maxima = 27.0
Error validacion = 24.505153625873717
Numero de iteración = 1595, parametro criterio = mse, parametro min_samples_split = 1.0, parametro min_samples_leaf = 0.5, parametro max_features = log2, parametro profundidad maxima = 28.0
Error validacion = 24.505153625873717
Numero de iteración = 1596, parametro criterio = mse, parametro min_samples_split = 1.0,

Error validacion = 19.68419003115265
Numero de iteración = 1629, parametro criterio = mae, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 30.0
Error validacion = 19.68419003115265
Numero de iteración = 1630, parametro criterio = mae, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 31.0
Error validacion = 19.68419003115265
Numero de iteración = 1631, parametro criterio = mae, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = sqrt, parametro profundidad maxima = 32.0
Error validacion = 19.68419003115265
Numero de iteración = 1632, parametro criterio = mae, parametro min_samples_split = 0.1, parametro min_samples_leaf = 0.1, parametro max_features = log2, parametro profundidad maxima = 1.0
Error validacion = 22.225467289719628
Numero de iteración = 1633, parametro criterio = mae, p

KeyboardInterrupt: 

Mostramos la tabla del error de validacion segun los diferentes parametros para elegir el modelo que mejor se ajuste a nuestros datos

In [43]:
error_val.sort_values(by=['val_error']).head(10)

,val_error,criterion,min_samples_split,min_samples_leaf,max_features,max_depth
1603,19.650312,mae,0.1,0.1,sqrt,4.0
1602,19.662967,mae,0.1,0.1,sqrt,3.0
1614,19.684190,mae,0.1,0.1,sqrt,15.0
1629,19.684190,mae,0.1,0.1,sqrt,30.0
1628,19.684190,mae,0.1,0.1,sqrt,29.0
1627,19.684190,mae,0.1,0.1,sqrt,28.0
1626,19.684190,mae,0.1,0.1,sqrt,27.0
1625,19.684190,mae,0.1,0.1,sqrt,26.0
1623,19.684190,mae,0.1,0.1,sqrt,24.0
1622,19.684190,mae,0.1,0.1,sqrt,23.0


Elegimos el modelo que menor error de validacion tenga

In [44]:
ind_min = error_val['val_error'].idxmin()
best_parameters = error_val.iloc[ind_min]
best_parameters

val_error            19.650312
criterion                  mae
min_samples_split          0.1
min_samples_leaf           0.1
max_features              sqrt
max_depth                  4.0
Name: 1603, dtype: object

Reentrenamos el modelo con los parámetros óptimos y testeamos

In [45]:
model = DecisionTreeRegressor(criterion = best_parameters['criterion'],
                              min_samples_split = best_parameters['min_samples_split'],
                              min_samples_leaf = best_parameters['min_samples_leaf'],
                              max_features = best_parameters['max_features'],
                              max_depth = best_parameters['max_depth'],
                             random_state = 1)

Juntamos train y val para entrenar el modelo final

In [46]:
print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))
print('Validation data size = ' + str(X_val.shape))
print('Validation target size = ' + str(y_val.shape))

# Combinar train y validación
X_train = pd.concat((X_train,X_val), axis = 0)
y_train = np.concatenate((y_train, y_val), axis = 0)

print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))

Train data size = (23969, 72)
Train target size = (23969,)
Validation data size = (5136, 72)
Validation target size = (5136,)
Train data size = (29105, 72)
Train target size = (29105,)


In [47]:
## Ponemos un contador para ver cuanto tarda cada modelo
start = timeit.default_timer()

model.fit(X_train, np.array(y_train))

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  5.309071299999999


In [48]:
start = timeit.default_timer()

pred_train = model.predict(X_train)
pred_val = model.predict(X_val)
pred_test = model.predict(X_test)

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  0.01375779999997917


In [49]:
# Calcular métricas de evaluación
mae_train = mae(y_train, pred_train);
mae_val = mae(y_val, pred_val);
mae_test = mae(y_test, pred_test);

mse_train = mse(y_train, pred_train);
mse_val = mse(y_val, pred_val);
mse_test = mse(y_test, pred_test);

mape_train = mape(y_train, pred_train);
mape_val = mape(y_val, pred_val);
mape_test = mape(y_test, pred_test);

In [50]:
print('Train: MAE = ' + str(mae_train) + ' - MSE = '  + str(mse_train) + ' - MAPE =' + str(mape_train))
print('Validation: MAE = ' + str(mae_val) + ' - MSE = '  + str(mse_val) + ' - MAPE =' + str(mape_val))
print('Test: MAE = ' + str(mae_test) + ' - MSE = '  + str(mse_test) + ' - MAPE =' + str(mape_test))

Train: MAE = 22.687167153410066 - MSE = 866.2586497165436 - MAPE =17.41202205933334
Validation: MAE = 19.27609034267913 - MSE = 634.5802180685358 - MAPE =14.332436741967406
Test: MAE = 23.653883589643762 - MSE = 968.714424761534 - MAPE =18.1607932362953


# Importancia de Variables